# Optic Nerve Registration
## Non-linear movement/distortion correction in human optic nerve diffusion imaging

### Steps:
* extract optic nerve centerline
* fit the double Gaussian optic nerve model to coronal slices
* segment optic nerve
* register segmentation
* apply warp

## USER INPUT: directory / filename

In [4]:
working_dir = '/Users/joowon/research/optic_nerve/DBSI_ON/DBSIONE008/16_0128/preproc_12_15/DBSIONE008_reg_wo_eddy'
filename = 'DBSIONE008_PA_2merged_rigid.nii.gz'

### import modules

In [5]:
%matplotlib inline
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import os
import sys
from on_utils import filename_wo_ext
import scipy.optimize as opt
import math
import time
import on_dbsi_utils
import on_model

In [7]:
filename_on_init = filename_wo_ext(filename) + '_on_init.nii.gz'
filename_flt_1 = filename_wo_ext(filename) + '_fltered_1.nii.gz'
filename_flt = filename_wo_ext(filename) + '_fltered.nii.gz'
filename_edge = filename_wo_ext(filename) + '_fltered_edge.nii.gz'
filename_centerline = filename_wo_ext(filename) + '_on_centerline.nii.gz'
filename_centerline_dilated = on_dbsi_utils.filename_wo_ext(filename) + '_on_centerline_dilated.nii.gz'
filename_model = filename_wo_ext(filename) + '_on_model.nii.gz'

## Run centerline extraction

## Check and modify centerline

## Fitting

In [9]:
on = on_model.OpticNerveFit()
on.read_dat(os.path.join(working_dir, filename))
on.read_init_center(os.path.join(working_dir, filename_centerline))
on.fit()

/usr/local/lib/python2.7/site-packages/scipy/optimize/minpack.py:690: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


--- 298.088864088 seconds ---


In [11]:
on.save(os.path.join(working_dir, 'tmp_on_save'))

### Find fitting outliers and estimate correct centers

In [12]:
on.outlier()
on.estimate()

In [16]:
on.save(os.path.join(working_dir, 'tmp_on_save_2'))

In [18]:
on_dil = on.make_segmentation()

In [19]:
on.hdr.set_data_dtype(on_dil.dtype)
img_tmp = nib.Nifti1Image(on_dil, on.aff, on.hdr)
nib.save(img_tmp, os.path.join(working_dir, filename_model))

## Registration

In [85]:
run_command = on_dbsi_utils.run_command

In [86]:
fn_in = os.path.join(working_dir, filename)
fn_model = os.path.join(working_dir, filename_model)

dn_reg = os.path.join(working_dir, 'registration')
fn_out_base = os.path.join(dn_reg, filename_wo_ext(filename))

run_command('mkdir -p %s' % dn_reg)

>> mkdir -p /Users/joowon/research/optic_nerve/DBSI_ON/DBSIONV006/15_1118/preproc_16_18/DBSIONV006_reg/registration


0

In [87]:
verbose = False
for f in range(dat.shape[-1]):
    fn_frame = '%s_on_model_%03d.nii.gz' % (fn_out_base, f)
    cmd = 'fslroi %s %s %s 1' % (fn_model, fn_frame, f)
    run_command(cmd, verbose)
    
    fn_frame = '%s_frame_%03d.nii.gz' % (fn_out_base, f)
    cmd = 'fslroi %s %s %s 1' % (fn_in, fn_frame, f)
    run_command(cmd, verbose)
    

fn_ref_on_model = '%s_on_model_%03d.nii.gz' % (fn_out_base, 0)
fn_ref = '%s_on_model_%03d.nii.gz' % (fn_out_base, 0)
run_command('cp -f %s_frame_%03d.nii.gz %s_frame_%03d_xenc.nii.gz' % (fn_out_base, 0, fn_out_base, 0), verbose)
run_command('cp -f %s_on_model_%03d.nii.gz %s_on_model_%03d_xenc.nii.gz' % (fn_out_base, 0, fn_out_base, 0), verbose)

for f in range(1, dat.shape[-1]):
    fn_frame = '%s_on_model_%03d.nii.gz' % (fn_out_base, f)
    fn_warp = '%s_on_model_%03d_warp' % (fn_out_base, f)
    fn_out = '%s_on_model_%03d_xenc.nii.gz' % (fn_out_base, f)
    cmd = on_dbsi_utils.reg_ms(fn_ref_on_model, fn_frame, fn_warp, fn_out, fn_mask=None)
    run_command(cmd, verbose)
    
    fn_frame = '%s_frame_%03d.nii.gz' % (fn_out_base, f)
    fn_out = '%s_frame_%03d_xenc.nii.gz' % (fn_out_base, f)
    cmd = 'antsApplyTransforms -d 3 -i %s -r %s -o %s -n BSpline[3] -t %s1Warp.nii.gz' \
        % (fn_frame, fn_ref, fn_out, fn_warp)
    run_command(cmd, verbose)
    
    

### Merge registered frames

In [109]:
fn_reg = '%s_reg.nii.gz' % fn_out_base
fns_frame = ['%s_frame_%03d_xenc.nii.gz' % (fn_out_base, f) for f in range(dat.shape[-1])]
cmd = 'fslmerge -t %s %s' % (fn_reg, ' '.join(fns_frame))
run_command(cmd, verbose)

0

### Merge registered centerlines (optional)

In [108]:
fn_model_reg = '%s_on_model_reg.nii.gz' % fn_out_base
fns_frame = ['%s_on_model_%03d_xenc.nii.gz' % (fn_out_base, f) for f in range(dat.shape[-1])]
cmd = 'fslmerge -t %s %s' % (fn_model_reg, ' '.join(fns_frame))
run_command(cmd, verbose)

0